In [45]:
import requests
import pandas as pd
from dotenv import load_dotenv
load_dotenv('../05_data_scraping/.env')
import os
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()
from dbio import mood_to_db, db_connect
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import string
import koreanize_matplotlib
import time
from konlpy.tag import Mecab
mecab = Mecab()
from nltk.probability import FreqDist
from wordcloud import WordCloud
import re
from datetime import datetime

## 키워드 형태소 분리 안되게 하기 - 최종 - 오피셜
- 대신 api 개수 조절하기

In [56]:
def w2v_4(keyword) :
    client_id = os.getenv('client_id') # 환경 변수에 접근
    client_pw = os.getenv('client_pw') # 환경 변수에 접근

    stopwords = pd.read_csv('https://raw.githubusercontent.com/haram4th/ablearn/main/%ED%95%9C%EA%B5%AD%EC%96%B4%EB%B6%88%EC%9A%A9%EC%96%B4100.txt', header = None)
    ex_stop_words = ['샤람']
    for i in ex_stop_words : 
        stopwords = stopwords.drop(stopwords[stopwords[0] == i].index)
    stopwords.reset_index(drop=True, inplace=True)
    
    url = "https://openapi.naver.com/v1/search/blog.json"
    final_data_keyword = pd.DataFrame()
    start = time.time() # 시작
    for i in range(1, 5) :
        payload = {'query' : keyword, 'display' : 100, 'start' : i, 'sort' : 'sim'}
        headers = {'X-Naver-Client-Id' : client_id, 'X-Naver-Client-Secret' : client_pw}
        r = requests.get(url, params = payload, headers = headers)
        if r.status_code == 200 :
            data = r.json()
        else :
            print("Error Code:", r.status_code)
            
        df_keyword_sim = pd.json_normalize(data['items'])
        entire_data_keyword_sim = make_df(df_keyword_sim)
        final_data_keyword = pd.concat([final_data_keyword, entire_data_keyword_sim], ignore_index=True)
        # 형태소 분석 후 후처리
    def process_title(title, keyword):
        # '망리단길'을 임시 토큰으로 변경
        title = title.replace(keyword, '우주')
        # 형태소 분석
        words = mecab.pos(re.sub(r"[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "", title))
        # 임시 토큰을 원래의 '망리단길'로 복원
        processed_words = [(word[0].replace('우주', keyword), word[1]) for word in words]
        return processed_words
        
    # DataFrame에 적용
    final_data_keyword['title2'] = final_data_keyword['title'].apply(lambda x : process_title(x, keyword))

    target = ['NNG', 'NNP', 'MAG', 'XR', 'NR', 'VV', 'VA', 'VCP', 'VCN', 'MM', 'IC', 'XPN', 'UNKNOWN']

    # 형태소에서 특정 단어 제거 및 태그 필터링
    def extract_words(words_list) : 
        return [word for word, pos in words_list if pos in target]
    final_data_keyword['filtered'] = final_data_keyword['title2'].apply(lambda x : extract_words(x))
    final_data_keyword['filtered_list'] = final_data_keyword['filtered'].apply(lambda x : eval(x) if isinstance(x, str) else x)
    from gensim.models import Word2Vec
    model_keyword_pos = Word2Vec(final_data_keyword['filtered_list'], vector_size=100, window=5, min_count=2, workers=8, sg=0)
    model_keyword_pos_result = model_keyword_pos.wv.most_similar(keyword, topn = 50)
    print(f"{time.time()-start:.4f} sec")
    return keyword, model_keyword_pos_result

In [57]:
mangri_4 = w2v_4('망리단길')

                                   title  \
0         늦게까지 하는 망리단길 손맛 대장 망원술집 :  휴식당   
1       산볼 - 망리단길 점메추, 가볍지만 근본 있는 일본식 덮밥   
2                    서울시장투어 망원시장 망리단길 포차   
3                망리단길 맛집 타코닷, 망원동 타코 파히타   
4            합정 분위기 좋은 망리단길 맛집 슬로우야드 바베큐   
..                                   ...   
395         [망원 소품샵] 수바코 본점 망리단길 일본 빈티지샵   
396  [내돈내산 맛집] 망원동/망리단길 벨라 또띠아 :: 난시안적 삶   
397         망리단길 술집 블렌딩 바 망원 훈남 사장님 친절해요   
398      망원동 찐맛집 추천 망리단길데이트코스 추천 :: 와들와들   
399          망리단길 술집 망원시장 근처 분위기 좋은 술잔망원   

                                                  link  \
0          https://blog.naver.com/rui-zzi/223653729408   
1     https://blog.naver.com/meandmeowsuk/223600715833   
2        https://blog.naver.com/bingohwan/223628357562   
3            https://blog.naver.com/hatvo/223582279017   
4     https://blog.naver.com/sksms1212111/223635324058   
..                                                 ...   
395       https://blog.naver.com/jinjinjw/223638339442   
396    

In [58]:
mangri_4

('망리단길',
 [('망원', 0.9530227780342102),
  ('맛집', 0.9478905200958252),
  ('망원동', 0.9433488249778748),
  ('카페', 0.9344775080680847),
  ('좋', 0.9162643551826477),
  ('분위기', 0.9084247350692749),
  ('술집', 0.9053659439086914),
  ('샵', 0.9026827216148376),
  ('데이트', 0.9024482369422913),
  ('돈', 0.8973955512046814),
  ('있', 0.8912801146507263),
  ('탑', 0.8815398812294006),
  ('바', 0.8789120316505432),
  ('서울', 0.8778261542320251),
  ('블', 0.8772179484367371),
  ('렌', 0.8742979764938354),
  ('르', 0.8733153939247131),
  ('소품', 0.8711498379707336),
  ('커피', 0.8705149292945862),
  ('후기', 0.859917163848877),
  ('추천', 0.8594532012939453),
  ('먹', 0.8587040305137634),
  ('까스', 0.8556848764419556),
  ('육회', 0.8534823656082153),
  ('빈티지', 0.8510558605194092),
  ('혼', 0.8451012969017029),
  ('딩', 0.8428353071212769),
  ('함께', 0.8426253199577332),
  ('거스', 0.8407586812973022),
  ('코스', 0.8376424312591553),
  ('가득', 0.8364389538764954),
  ('일본', 0.8345228433609009),
  ('다', 0.833975613117218),
  ('ㅣ', 0.83

In [60]:
hongdae_4 = w2v_4('홍대거리')
hongdae_4

                                 title  \
0               서울여행(홍대거리, 일산, 대학교 투어)   
1      홍대 술집 거리 도른계 분위기 좋은 핫플 치킨 맛집 추천   
2                내 안의 소년 그리고 눈내리던 홍대거리   
3            홍대입구 고기집 신선화로 주말 홍대거리 즐기기   
4      홍대거리에 위치한 서울시 교육청 마포평생학습관과 마포리움   
..                                 ...   
395                홍대간판 - 홍대거리의 파사드 모음   
396                          홍대 거리 미술들   
397  서울 홍대 차 없는 거리에 위치한 널디플래그쉽 스토어...    
398         홍대거리칵테일 종류 다양한 홍대감성술집 락소울윙   
399      홍대 술집 추천 합리적이였던 홍대 술집 거리 탐라육해   

                                                 link  \
0        https://blog.naver.com/juyahong/223505978347   
1         https://blog.naver.com/ssso131/223540235854   
2        https://blog.naver.com/sawtiger/223299175787   
3         https://blog.naver.com/ojiknar/223387464802   
4          https://blog.naver.com/yeojh1/223611747063   
..                                                ...   
395   https://blog.naver.com/forhomedeco/140127756368   
396     https://blog.naver.com/tomatone

('홍대거리',
 [('홍대', 0.7997704148292542),
  ('거리', 0.7960033416748047),
  ('레드', 0.7207735180854797),
  ('추천', 0.715246856212616),
  ('이', 0.7119947075843811),
  ('성수역', 0.7097966074943542),
  ('로드', 0.6606968641281128),
  ('맛집', 0.6576990485191345),
  ('출구', 0.6451554894447327),
  ('볼거리', 0.6444752812385559),
  ('카페', 0.6311955451965332),
  ('경의선', 0.6282651424407959),
  ('투어', 0.6239126324653625),
  ('공연', 0.6127001643180847),
  ('상수', 0.6066823601722717),
  ('야', 0.5989619493484497),
  ('마포', 0.5987105369567871),
  ('서울', 0.5945063829421997),
  ('인파', 0.5926302075386047),
  ('새', 0.5911649465560913),
  ('책거리', 0.579802930355072),
  ('위치', 0.575829803943634),
  ('쇼룸', 0.5683109760284424),
  ('젊음', 0.5609431266784668),
  ('복잡', 0.5601528882980347),
  ('홍대입구역', 0.560018002986908),
  ('숲길', 0.5592488050460815),
  ('라인업', 0.5578824281692505),
  ('스토어', 0.5441254377365112),
  ('좋', 0.5436860918998718),
  ('축제', 0.5436516404151917),
  ('쉐', 0.5416958332061768),
  ('차', 0.5405667424201965),
  

In [61]:
songri_4 = w2v_4('송리단길')
songri_4

                                  title  \
0              송리단길카페 오쿠쿠베이커리 잠실 루프탑 카페   
1               잠실 송리단길 방이동 막창 맛집 송파 단막   
2           잠실 송리단길 맛집 주차가능한 소고기맛집 숯불성화   
3                 쌀쌀해질 때 송리단길 우동 요미우돈교자   
4         잠실 소고기 내돈내산 맛집▫송리단길 숯불성화 무료주차   
..                                  ...   
395  송리단길에서 데이트하다 내... 식당을 만난 이야기 송리단취향   
396          송리단길 맛집 : 토닭토닭, 괜찮았던 닭갈비맛집   
397               송리단길 술집 라쿤스드링크 분위기 맛집   
398          송리단길 맛집 | 다케오호르몬데판야끼 송리단길점   
399           잠실맛집 메밀집 내돈내산 찐후기 #송리단길맛집   

                                                 link  \
0      https://blog.naver.com/lovely_227/223658567426   
1           https://blog.naver.com/ck_mh/223658569382   
2      https://blog.naver.com/shinyg1022/223653828006   
3           https://blog.naver.com/tarbi/223630095665   
4        https://blog.naver.com/conytrip/223659695013   
..                                                ...   
395     https://blog.naver.com/yoorikong/223595358289   
396        https://blog.nav

('송리단길',
 [('맛집', 0.9764217138290405),
  ('잠실', 0.9521239399909973),
  ('좋', 0.947024941444397),
  ('카페', 0.938275158405304),
  ('석촌', 0.9355560541152954),
  ('집', 0.9337067008018494),
  ('후기', 0.9311211109161377),
  ('솥', 0.926596462726593),
  ('돈', 0.9157143831253052),
  ('디저트', 0.9129819869995117),
  ('데이트', 0.9117943644523621),
  ('분위기', 0.9038130044937134),
  ('유포', 0.9035336375236511),
  ('송파', 0.9003883004188538),
  ('솔', 0.8997930884361267),
  ('맥주', 0.8952676653862),
  ('술집', 0.8928651213645935),
  ('씽', 0.8915162086486816),
  ('리프', 0.8913477063179016),
  ('빵', 0.8911872506141663),
  ('송리', 0.8905112147331238),
  ('식당', 0.8873624205589294),
  ('단', 0.8862679600715637),
  ('호수', 0.8834539651870728),
  ('백', 0.8827444911003113),
  ('근처', 0.8826975226402283),
  ('과일', 0.882619321346283),
  ('립스', 0.8807868361473083),
  ('추천', 0.8762301206588745),
  ('즐기', 0.8744615912437439),
  ('버슬', 0.8741577863693237),
  ('재', 0.8699032664299011),
  ('게장', 0.8679477572441101),
  ('매듭', 0.8654